In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 5.5
fig_height = 3.5
fig_format = 'pdf'
fig_dpi = 300

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/francoisderyckel/Documents/quant-sandbox/python':
  os.chdir(r'/Users/francoisderyckel/Documents/quant-sandbox/python')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


/var/folders/4k/sh_jysz50kq5c92jvljhs06c0000gp/T/ipykernel_81754/3484563495.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/_bootstrap.py": 1701988380.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/_bootstrap_external.py": 1701988380.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/zipimport.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/codecs.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/encodings/aliases.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/encodings/__init__.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/encodings/utf_8.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/abc.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/io.py": 1701988379.0, "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/stat.py": 1701988379.0, "/Libr

In [2]:
#| label: loading-data
#| warning: false
#| output: false

# Loading the required library 
import pandas as pd
import numpy as np
import yfinance as yf

rio = yf.download('RIO')
df = pd.DataFrame(rio)

[*********************100%%**********************]  1 of 1 completed

In [3]:
#| label: initial-eda

# get to know df variables
print('The shape of dataframe is:', df.shape)
print(df.head())

# checking for missing values. 
df.isnull().sum()

The shape of dataframe is: (8542, 6)
                Open      High       Low     Close  Adj Close  Volume
Date                                                                 
1990-06-28  10.09375  10.09375   9.96875   9.96875   1.916216  176400
1990-06-29  10.03125  10.06250  10.00000  10.06250   1.934236   69200
1990-07-02  10.00000  10.03125  10.00000  10.03125   1.928229   62000
1990-07-03  10.03125  10.06250  10.03125  10.06250   1.934236   29600
1990-07-05   9.71875   9.71875   9.65625   9.68750   1.862152   31200


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [4]:
df[['Adj Close', 'Volume']].describe().round(2)

,Adj Close,Volume
count,8542.00,8542.00
mean,19.34,1705348.65
std,18.05,1853149.46
min,1.44,0.00
25%,4.00,60400.00
50%,16.13,1326300.00
75%,27.32,2836650.00
max,73.61,19088900.00


In [5]:
#| label: plot01

import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta

# Checking only over the last 5 years of data 
start_date = date.today() + relativedelta(days = int(np.round(-5 * 365)))
end_date = date.today()

# first vizualization 
plt.plot(df['Adj Close'].loc[start_date:end_date])
plt.show()

<Figure size 1650x1050 with 1 Axes>

In [6]:
#| label: candlestick-plot01

import mplfinance as mpl

start_date = date.today() + relativedelta(days = int(np.round(-0.5 * 365)))

df_last18months = df.loc[start_date:end_date]

fig, axlist = mpl.plot(
  df_last18months, 
  type = 'candle', 
  style='binance', 
  figratio = (15, 7), 
  returnfig = True
)

fig = fig.suptitle(
  f"$RIO stock between {start_date} and {end_date}", 
  y=.95, 
  fontsize=15, 
  weight = 'semibold', 
  style = 'normal'
)

mpl.show()

<Figure size 1232.14x575 with 2 Axes>

In [7]:
#| label: feature-eng-1

# getting returns. The first one we have shifted it as it constitutes the base of what we have to predict. 
df['ret_1d_target'] = np.log(df['Adj Close'].shift(-1) / df['Adj Close'])
df['ret_1d_shifted'] = (np.log(df['Adj Close']).diff(1)).shift(1)

# Creating new variables for intraday 
df['o-c'] = (df.Open - df.Close) / df.Close
df['h-l'] = (df.High - df.Low) / df.Close

df.head()

# creating our lag returns 
for lags in range(2, 19, 1): 
  df['ret_1d_lag' + str(lags) + 'd'] = df.ret_1d_shifted.shift(lags)

for days in range(2, 65, 5): 
  df['ret_'+ str(days)] = df.ret_1d_shifted.rolling(days).sum()
  df['sd_' + str(days)] = df.ret_1d_shifted.rolling(days).std()

for lags in range(2, 65, 5): 
  df['sma_' + str(lags) + 'd'] = df['Adj Close'].rolling(window=lags).mean()
  df['sma_' + str(lags) + 'd'] = np.log(df['Adj Close'] / df['sma_' + str(lags) + 'd'])
  df['sma_' + str(lags) + 'd'] = df['sma_' + str(lags) + 'd'].shift(1)
  
for lags in range(3, 65, 5): 
  df['ema_' + str(lags) + 'd'] = df['Adj Close'].ewm(span = lags, adjust = False).mean() 
  df['ema_' + str(lags) + 'd'] = np.log(df['Adj Close'] / df['ema_' + str(lags) + 'd'])
  df['ema_' + str(lags) + 'd'] = df['ema_' + str(lags) + 'd'].shift(1)


In [8]:
#| label: create-target
df2 = df.copy()

df.dropna(inplace = True)

# Only use last 5 years of data as suggested by exam instructions
df = df.loc['2019-01-01':]

#We will use the median returns to ensure we have balanced class 

df['ret_1d_shifted'].describe()

# find median
median_return = df['ret_1d_shifted'].dropna().median()
df['target'] = np.where(df['ret_1d_target'] > median_return, 1, 0)
df['target'] = df['target'].shift(-1)

# checking we have the right ratio. 
df['target'].value_counts(normalize = True).round(4) * 100

target
1.0    50.04
0.0    49.96
Name: proportion, dtype: float64

In [9]:
# removing NaN values
df.dropna(inplace = True)

# Only use last 5 years of data as suggested by exam instructions
df = df.loc['2019-01-01':]

# checking we have the right ratio. 
df['target'].value_counts(normalize = True).round(4) * 100

target
1.0    50.04
0.0    49.96
Name: proportion, dtype: float64

In [10]:
#| label: training-testing

X = df.drop(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'target',
               'ret_1d_target'], axis = 1).values
y = df['target'].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

print(f"Size of training set is {len(x_train)} and size of testing set is {len(x_test)}")

Size of training set is 1085 and size of testing set is 272


In [11]:
#| label: base-model

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC 

model_svm_base = Pipeline([
  #('Scaler', MinMaxScaler()), 
  ('std_scaler', StandardScaler()), 
  ("pca", PCA(n_components = 0.95)),
  ('classifier', SVC(kernel = 'linear', probability = True))
])

model_svm_base.fit(x_train, y_train)
#x_train_transf = Pipeline.fit_transform(model_svm_lin) 

y_pred = model_svm_base.predict(x_test)

In [12]:
from sklearn.metrics import roc_curve, ConfusionMatrixDisplay, accuracy_score, auc

acc_train = accuracy_score(y_train, model_svm_base.predict(x_train))
acc_test = accuracy_score(y_test, y_pred)

print(acc_train)
print(acc_test)

0.5364055299539171
0.47794117647058826


In [13]:
conf_mat = ConfusionMatrixDisplay.from_estimator(
  model_svm_base, x_train, y_train, cmap = plt.cm.Blues
)
plt.title('Confusion Matrix on Base Model')
plt.show()

<Figure size 640x480 with 2 Axes>

In [14]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.50      0.53      0.51       141
         1.0       0.45      0.42      0.44       131

    accuracy                           0.48       272
   macro avg       0.48      0.48      0.48       272
weighted avg       0.48      0.48      0.48       272



In [15]:
from sklearn.metrics import roc_curve, auc

y_prob = model_svm_base.predict_proba(x_test)[:, 1]


fpr, tpr, threshold = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print(roc_auc)

plt.clf()
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

0.5260408207460344


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 1 Axes>

In [16]:
#| label: hyper-param
#| eval: false


from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, cross_val_score

tscv = TimeSeriesSplit(n_splits = 10, gap = 1)

param_grid = {'C': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 20], 
              'gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 20], 
              'kernel': ['rbf', 'poly', 'linear']
}

grid = GridSearchCV(SVC(), param_grid, verbose = 1, 
                    scoring = 'roc-auc', cv = tscv, n_jobs = 3)
grid.fit(x_train, y_train)

grid.best_params_
grid.best_score_
#grid.cv_results_

In [17]:
#| eval: false

svm_best_param = SVC(**grid.best_params_)
svm_best_param.fit(x_train, y_train, 
                   #eval_set = [(x_train, y_train), (x_test, y_test)], 
                   #verbose = True
                   )
#x_val_score = cross_val_score(svm_best_param, x_train, y_train, cv = tscv)

# what does this do?  
x_train_tuned = svm_best_param.decision_function(x_train)
x_val_score 

In [18]:
#| eval: false

y_pred = svm_best_param.predict(x_test)

acc_tuned_train = accuracy_score(y_train, svm_best_param.predict(x_train))
acc_tuned_test = accuracy_score(y_test, y_pred)

print(acc_train)
print(acc_test)